# Initialization

In [ ]:
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy, json
from helper import singleglob, json_merge
import events_methods
import itables


In [ ]:

itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
# itables.options.scrollY="200px"
# itables.options.scrollCollapse=True
# itables.options.paging=False
itables.options.column_filters = "footer"
itables.options.layout={"topEnd": None, "top1": "searchBuilder"}

In [ ]:
# base = Path("/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files/Rats/Test_Julien_ForcedInput/")
base = Path("/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files/Rats/Luisa/Rat101_0729_opto_01")
# base = Path("/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files/Humans/BAGOSMOV/EX1/")
dat_path = singleglob(base, "*.dat")
task_path = singleglob(base, "*.xls")
info_path = singleglob(base, "*.yaml", search_upward_limit=Path("/home/julienb/Documents/database_scripts/database_scripts_test/poly_dat_files"))
res_events_path = base/"events.tsv"
info_path

# Reading poly events and adding task information 

In [ ]:
event_df = pd.read_csv(dat_path, sep="\t", names=['time (ms)', 'family', 'nbre', '_P', '_V', '_L', '_R', '_T', '_W', '_X', '_Y', '_Z'], skiprows=13, dtype=int)
event_df.insert(0, "t", event_df.pop("time (ms)")/1000)
event_df = event_df.reset_index(names="poly_evnum").sort_values(["t", "poly_evnum"]).reset_index(drop=True)
event_df["task_node"] = event_df["_T"].where(event_df["family"]==10).ffill()
event_df

In [ ]:
task_df = pd.read_csv(task_path, sep="\t", header=11)
task_df = task_df.rename(columns={task_df.columns[0]: "task_node" })
display(task_df.columns)
task_df

In [ ]:


channels = pd.Series(task_df.columns).str.extract(r'\s*(?P<channel_name>\w+)\s*\((?P<family>\d+)\s*,\s*(?P<nbre>\d+)\)\s*').assign(taskcol_name=task_df.columns).dropna(how="any")
channels["family"] = channels["family"].astype(int)
channels["nbre"] = channels["nbre"].astype(int)
channels

    

In [ ]:
pattern=r'on\(\d+(,\d+)*\)'
task_info=pd.DataFrame()
stacked = task_df.set_index("task_node")[channels["taskcol_name"].to_list()].stack().str.lower().str.strip().dropna()
stacked.index.names=["task_node", "taskcol_name"]
task_info["data"] = stacked
task_info["match"] = task_info["data"].str.fullmatch(pattern)
task_info = task_info.loc[task_info["match"]]
task_info["important"] = task_info["data"].str.slice(3, -1)
task_info["task_params"] = task_info["important"].str.split(",").apply(lambda l: [float(x) for x in l])
task_info = task_info.drop(columns=["important", "match", "data"]).join(stacked.rename("task_data"), how="outer")
task_info = task_info.reset_index() 
task_info["task_node"] = task_info["task_node"].astype(float)
task_info


In [ ]:
event_channels_df = channels.merge(event_df, on=["family", "nbre"], how="right").merge(task_info, on=["taskcol_name", "task_node"], how="left").sort_values("t")
event_channels_df

# Extracting configuration information

In [ ]:
info = yaml.safe_load(info_path.open("r"))


In [ ]:
event_spec = events_methods.FiberEventProcessing.process_info(channels["channel_name"].to_list(), info["processing"])
pd.DataFrame(list(event_spec.values()))


# Running event extraction methods

In [ ]:
event_channels_df = event_channels_df.sort_values(["t", "poly_evnum"])
all=[]
for ev_name, item in event_spec.items():
    ev_dataframe = events_methods.PolyEventProcessing.compute_evdataframe(event_channels_df, item)
    if len(ev_dataframe.index) == 0: continue
    events = events_methods.PolyEventProcessing.call(item["method"],ev_dataframe, item)
    all.append(events.reset_index(drop=True))

all = pd.concat(all).sort_values("t")
all
  

# Renaming and exporting, displaying reload

In [ ]:
if "display" in info and "rename" in info["display"]:
    all["event_name"] = all["event_name"].map(lambda e: info["display"]["rename"][e] if e in info["display"]["rename"] else e)
json_cols = ["metadata", "waveform_changes", "waveform_values"]
for col in json_cols:
    all[f"{col}_json"] = all[col].apply(lambda d: json.dumps(d))
all.drop(columns=json_cols).to_csv(res_events_path, sep="\t", index=False)
reloaded = pd.read_csv(res_events_path, sep="\t", index_col=False)
for col in reloaded.columns:
    if col.endswith("_json"):
        reloaded[col[:-5]] = reloaded.pop(col).apply(lambda s: json.loads(s) if not pd.isna(s) else None)
reloaded

# Summary Information (Checks)

In [ ]:
summary = reloaded.copy()

def compute_grp(duration):
    n = len(duration.index)
    rounded = np.round(duration*100)/100
    counts = rounded.value_counts()
    important = (counts > 2) & (counts > n/5)
    important_min_val = counts.loc[important].index.min()
    important_max_val = counts.loc[important].index.max()
    ret = np.where(rounded.map(important), rounded, 
          np.where(duration<important_min_val, f"{rounded.min()}<=v<{important_min_val}", 
          np.where(duration>important_max_val, f"{important_max_val}<v<={rounded.max()}", 
          "_"
        )))
    
    return ret


summary["duration_goup"]= summary.groupby(["event_name", "n_segments"])["duration"].transform(compute_grp)
summary["metadata"].iat[0] = dict(warnings=dict(test="2", other="3"))
warning_list = summary["metadata"].apply(lambda d: list(d["warnings"].keys()) if not d is None and "warnings"in d else [])
all_warnings = set(warning_list.sum())

def compute_info(grp):
    warnings = {k: 0 for k in all_warnings}
    for m in grp["metadata"].values:
        if m is None: continue
        if "warnings" in m:
            for k in m["warnings"].keys():
                if not k in warnings:
                    warnings[k]=0
                warnings[k]+=1
    return pd.Series(dict(n=len(grp.index)) | {f'warning_{k}': v for k, v in warnings.items()})
summary.groupby(["event_name", "n_segments", "duration_goup"]).apply(compute_info, include_groups=False).reset_index()